### Goal:
Implement FULL CRUD for one resource
using clean structure and correct REST behavior


#### Fields
- uid (UUID)            → public identifier
- name (str)            → required
- price (float)         → required, > 0
- is_active (bool)      → default True
- created_at (datetime)


In [1]:
"""
    Resource: Product

"""

'\n    Resource: Product\n\n'

In [ ]:
"""
    1. Databse Model
"""

import uuid
from datetime import datetime
from sqlmodel import SQLModel, Field

class Product(SQLModel, table=True):
    __tablename__="product"
    uid: uuid.UUID | None = Field(default_factory=uuid.uuid4, primary_key=True, index=True)
    name: str = Field(unique=True, nullable=False, index=True)
    price: float = Field(nullable=False)
    is_active: bool = Field(default=True, nullable=False, index=True)
    created_at: datetime = Field(default_factory=datetime.now, nullable=False)

In [ ]:
"""
    2. Schema
"""
from datetime import datetime
from typing import Optional
from pydantic import BaseModel

'''Output Schema'''
class ProductRead(BaseModel):
    uid: uuid.UUID
    name: str
    price: float
    is_active: bool
    created_at: datetime
    
'''Input Schema'''
class ProductCreate(BaseModel):
    name: str
    price: float


class ProductPutUpdate(BaseModel):
    name: str
    price: float
    is_active: bool

class ProductPatchUpdate(BaseModel):
    name: Optional[str] = None
    price: Optional[float] = None
    is_active: Optional[bool] = None

In [ ]:
""" 
    3. Services
"""
import uuid
from sqlmodel import select
from sqlalchemy.ext.asyncio import AsyncSession

class ProductService:
    async def read_products(self, session: AsyncSession):
        statement = select(Product)
        result = await session.execute(statement)
        products = result.scalars().all()

        return products
    
    
    async def search_by_id(self, product_uid: uuid.UUID, session: AsyncSession):
        statement = select(Product).where(Product.uid == product_uid)
        result = await session.execute(statement)
        product = result.scalar_one_or_none()
        
        if not product:
            raise ValueError("Product not found")
        
        return product
    
    
    async def create_product(self, payload: ProductCreate, session: AsyncSession):
        statement = select(Product).where(Product.name == payload.name)
        result = await session.execute(statement)
        product = result.scalar_one_or_none()
        
        if product:
            raise ValueError("Product already exists!")
        
        new_product = Product(name=payload.name,
                              price=payload.price)
        
        session.add(new_product)
        await session.commit()
        await session.refresh(new_product)
        
        return new_product
    
    
    async def update_put_product(self, product_uid: uuid.UUID, payload: ProductPutUpdate, session: AsyncSession):
        product = await self.search_by_id(product_uid, session)
        
        if not product:
            raise ValueError("Product not found")
        
        product.name = payload.name
        product.price = payload.price
        product.is_active = payload.is_active
        
        session.add(product)
        await session.commit()
        await session.refresh(product)
        
        return product
    
    
    async def update_patch_product(self, product_uid: uuid.UUID, payload: ProductPatchUpdate, session: AsyncSession):
        product = await self.search_by_id(product_uid, session)
        
        if not product:
            raise ValueError("Product not found")
        
        data = payload.model_dump(exclude_unset=True)
        if not data:
            raise ValueError("No field provided to update")
        
        for field, value in data.items():
            setattr(product, field, value)
            
            
        session.add(product)
        await session.commit()
        await session.refresh(product)
        
        return product
        
    
    async def delete_product(self, product_uid: uuid.UUID, session:AsyncSession):
        product = await self.search_by_id(product_uid, session)
        
        if not product:
            raise ValueError("Product not found")
        
        if not product.is_active:
            return product
        
        product.is_active = False
        session.add(product)
        await session.commit()
        await session.refresh(product)

        return product

In [5]:
""" 
    4. Routes
"""

' \n    4. Routes\n'

In [6]:
"""
    5. Runing Fastapi file
"""

'\n    5. Runing Fastapi file\n'